In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from models.mida import Mida
from models.mice import Mice

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from fancyimpute import KNN, SoftImpute

from preprocessing.data_loading import DataSet
from preprocessing.missing_value_generation import mcar_generator, mar_generator, mnar_generator
from models.imputation_wrapper import SingleImputationWrapper, MultiImputationWrapper
from analysis.evaluation import OutlierEvaluation, LnormEvaluation, TimeEvaluation
from analysis.method_wrapper import OutlierMethodWrapper

In [58]:
dataset = DataSet('ctg', 'outlier')
amputation = mar_generator(2)
dataset.init_missing_data(amputation)

/home/marcus/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:2248: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [59]:
out_evaluation = OutlierEvaluation(OutlierMethodWrapper(LocalOutlierFactor()), dataset)
lnorm_evaluation = LnormEvaluation(dataset)
time_evaluation = TimeEvaluation(dataset)

In [60]:
# result on complete dataset
out_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')
lnorm_evaluation.evaluate_result([dataset.complete_data()], 0, 'None', 'None')

In [61]:
imputations = [SingleImputationWrapper(KNN(3, verbose=False)), SingleImputationWrapper(SoftImpute(verbose=False)),
               MultiImputationWrapper(Mice(40, verbose=False))]

In [62]:
#increase missing data rate
for p in [0.2, 0.4, 0.6, 0.8]:
    dataset.ampute_values(p)
    
    # apply each imputation
    for imputation in imputations:
        results, exec_time = imputation.complete(dataset.missing_data())
        out_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())
        lnorm_evaluation.evaluate_result(results, p, amputation.name(), imputation.name())
        time_evaluation.evaluate_result(exec_time, p, amputation.name(), imputation.name(), imputation.number())
        print(amputation.name() + ': ' + str(p) + ' ' + imputation.name() + ' done')

/home/marcus/Studium/Master/Semester_1/Smart Representation/Imputation/preprocessing/missing_value_generation.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return result


mar: 0.2 KNN done


/home/marcus/anaconda3/lib/python3.6/site-packages/fancyimpute-0.3.1-py3.6.egg/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars


mar: 0.2 SoftImpute done
mar: 0.2 Mice done
mar: 0.4 KNN done
mar: 0.4 SoftImpute done
mar: 0.4 Mice done
mar: 0.6 KNN done
mar: 0.6 SoftImpute done
mar: 0.6 Mice done
mar: 0.8 KNN done
mar: 0.8 SoftImpute done
mar: 0.8 Mice done


In [63]:
out_evaluation.evaluation_results

,"(settings, p)","(settings, mechanism)","(settings, imputation)","(settings, number)","(auc, post)","(auc, prev)"
0,0,None,None,1,0.632690,0.632690
1,0.2,mar,KNN,1,0.673407,0.673407
2,0.2,mar,SoftImpute,1,0.684853,0.684853
3,0.2,mar,Mice,1,0.686903,0.686903
4,0.2,mar,Mice,5,0.676394,0.675937
5,0.2,mar,Mice,10,0.669263,0.674664
6,0.2,mar,Mice,15,0.667760,0.674715
7,0.2,mar,Mice,20,0.660886,0.675213
8,0.2,mar,Mice,25,0.661343,0.677310
9,0.2,mar,Mice,30,0.658439,0.675718


In [64]:
lnorm_evaluation.evaluation_results

,"(settings, p)","(settings, mechanism)","(settings, imputation)","(settings, number)","(L1, post)","(L1, prev)","(L2, post)","(L2, prev)"
0,0,None,None,1,0.000000,0.000000,0.000000,0.000000
1,0.2,mar,KNN,1,16461.770594,16461.770594,86.534920,86.534920
2,0.2,mar,SoftImpute,1,9595.113517,9595.113517,59.261734,59.261734
3,0.2,mar,Mice,1,14652.130114,14652.130114,81.040795,81.040795
4,0.2,mar,Mice,5,14696.896599,13377.883898,81.269890,76.187808
5,0.2,mar,Mice,10,14700.656494,13035.614306,81.444801,75.644687
6,0.2,mar,Mice,15,14662.408753,12977.740445,81.315944,75.571198
7,0.2,mar,Mice,20,14683.027709,12969.197032,81.408878,75.888528
8,0.2,mar,Mice,25,14713.987743,12991.723322,81.635086,76.092795
9,0.2,mar,Mice,30,14768.539717,12965.336783,81.893902,76.023912


In [65]:
time_evaluation.evaluation_results

,p,mechanism,imputation,number,runtime
0,0.2,mar,KNN,1,1.414394
1,0.2,mar,SoftImpute,1,0.698897
2,0.2,mar,Mice,40,14.417942
3,0.4,mar,KNN,1,1.559932
4,0.4,mar,SoftImpute,1,0.493577
5,0.4,mar,Mice,40,13.268503
6,0.6,mar,KNN,1,1.735874
7,0.6,mar,SoftImpute,1,0.530869
8,0.6,mar,Mice,40,14.230292
9,0.8,mar,KNN,1,1.817914


In [70]:
test = list(out_evaluation.evaluation_results.T.to_dict().values())

In [76]:
pd.DataFrame.from_dict(test).groupby([('settings', 'number')])